**Overview**  
SemEval 2021 Task 8
Implemented NLP pipelines, to extract quantities, measurements along with their measured entities from scientific texts. (SemEval 2021 Task 8) More detailed info about the task can be found [here](https://github.com/harperco/MeasEval). 

So the given baseline model by the organizer were using Spacy NER model, and they were creating realtionships among them using a knockout logic. Their F1 overlap score was 0.21.

We first tried using CNN-biLSTM-CRF model implemented in [here](https://github.com/guillaumegenthial/tf_ner/tree/master/models/lstm_crf).

Using BERT we incresed it to 0.27
By changing the logic in the relation creation we increase it to 0.31.
Then using a BERT model trained on Scientific Texts( [SCIBERT](https://github.com/allenai/scibert)) we increased the f1 overlap score to 0.34. 


### Install Transformers Library

In [ ]:
pip install transformers -U

     |████████████████████████████████| 1.8MB 11.2MB/s 
     |████████████████████████████████| 890kB 49.2MB/s 
     |████████████████████████████████| 2.9MB 56.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=d1fef0395faa9d9d6a69d8e0bc47d4baa48730375d168b80b1ffc2316ef9cefb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
pip install watchdog==0.8.3

     |████████████████████████████████| 92kB 4.7MB/s 
  Created wheel for watchdog: filename=watchdog-0.8.3-cp36-none-any.whl size=72890 sha256=0e44a6eb70bba72c7f0fb7804c3518a899b341633e9e7bbe2dec890c078747e3
  Stored in directory: /root/.cache/pip/wheels/3b/a0/6f/8c52fea32c994f833b55e7b73078f81c155ec108011c04e708
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=f145179912786a9146a34c0607e719853f1e92134ba297bfaba5c0189b646e61
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built watchdog pathtools


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 204kB 9.8MB/s 
     |████████████████████████████████| 1.2MB 15.2MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 1.9MB 37.1MB/s 
     |████████████████████████████████| 317kB 59.2MB/s 
     |████████████████████████████████| 7.5MB 36.5MB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 133kB 50.4MB/s 
     |████████████████████████████████| 163kB 70.2MB/s 
     |████████████████████████████████| 4.5MB 38.4MB/s 
     |████████████████████████████████| 112kB 65.6MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 122kB 59.1MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=3114c94be56bac11ac91089ab19238b1a6dc56bb23b9d9ed65fb033dc93146af
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f

### Define functions Donwload Measeval data

In [ ]:
!git clone https://github.com/harperco/MeasEval.git

Cloning into 'MeasEval'...
remote: Enumerating objects: 3009, done.
remote: Counting objects: 100% (3009/3009), done.
remote: Compressing objects: 100% (2305/2305), done.
remote: Total 3009 (delta 657), reused 2979 (delta 628), pack-reused 0
Receiving objects: 100% (3009/3009), 1.18 MiB | 2.88 MiB/s, done.
Resolving deltas: 100% (657/657), done.


In [ ]:
import os
from os import walk
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pandas as pd
import string
from sklearn.model_selection import train_test_split
import shutil
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def getfile_bases(folder):
  ## read textfilenames
  txtnames = []
  textpath = ''
  if folder == 'trial':
    textpath = '/content/MeasEval/data/'+folder+'/txt/'
  else:
    textpath = '/content/MeasEval/data/'+folder+'/text/'

  for (dirpath, dirnames, filenames) in walk(textpath):
      txtnames.extend(filenames)
      break
  ## read tsvfilenames
  tsvnames = []
  for (dirpath, dirnames, filenames) in walk('/content/MeasEval/data/'+folder+'/tsv/'):
      tsvnames.extend(filenames)
      break
  ## list the mathced ones (having both tsv and txt file)
  b = []
  for f in filenames:
    base, ext = os.path.splitext(f)
    if base + '.tsv' in tsvnames :
      b.append(base)
  return b;

In [ ]:
def read_text_files(folder,base_list):
  textdata = []
  textpath = ''
  if folder == 'trial':
    textpath = '/content/MeasEval/data/'+folder+'/txt/'
  else:
    textpath = '/content/MeasEval/data/'+folder+'/text/'

  for base in base_list:
    f = open(textpath + base + '.txt', "r")
    text = f.read()
    f.close()
    textdata.append((base,text))
  X = pd.DataFrame(textdata, columns = ['docId', 'inputText'])
  return X

In [ ]:
def read_tsv_files(folder,base_list):
  dataframes = []
  for base in base_list:
    df = pd.read_csv('/content/MeasEval/data/'+folder+'/tsv/' + base + '.tsv',sep="\t")
    dataframes.append(df)
  Y = pd.concat(dataframes, ignore_index=True)
  return Y

In [ ]:
## removes the punctuations from words
def process_sentence(sentence):
 ## words = word_tokenize(sentence)
  words = sentence.split();
# words = sentence.split()
#  removelist = []
#  for w in words:
#    if w in string.punctuation:
#      removelist.append(w)
#  for i in removelist:
#    words.remove(i)
  return words

In [ ]:
## find unique indices for each word
def find_indices(inputText, words):
  words_with_indices = []
  startlist = []
  for word in words:
    x = inputText.find(word) 
    if x != -1:
      start = x
      end =  x + len(word)
      tmp = '!' * len(word)
      inputText = inputText[:start] + tmp + inputText[end:]
      startlist.append(str(start))
      words_with_indices.append((word,start,start+len(word)))
  return startlist,words_with_indices,inputText

In [ ]:
## find annottype of words w.r.t its index
def find_annotations(words_indices,labels):
  matched = [] 
  for i in words_indices:
    word = i[0]
    index = i[1]
    annotType = None
    for index2, row2 in labels.iterrows():
      start = row2['startOffset']
      end = row2['endOffset']
      if start <= index <= end:
 #       if row2['annotType'] == 'Quantity':
        annotType = row2['annotType']
       # else:
        #  annotType = "NT"
        break
    if annotType is None:
      matched.append('O')
    else:
      matched.append(annotType)
  return matched

In [ ]:
## find annottype of words w.r.t its index
def find_annotations_train(words_indices,labels):
  quantfound = False
  matched = [] 
  for i in words_indices:
    word = i[0]
    index = i[1]
    annotType = None
    for index2, row2 in labels.iterrows():
      start = row2['startOffset']
      end = row2['endOffset']
      if start <= index <= end:
        if row2['annotType'] == 'Quantity':
          quantfound = True
          annotType = row2['annotType']
        else:
          annotType = "NT"
        break
    if annotType is None:
      matched.append('NT')
    else:
      matched.append(annotType)
  if quantfound:
    return matched
  else:
    return None

In [ ]:
def create_sentences_tags(X,Y):
#  bases = getfile_bases()
#  X = read_text_files(bases)
#  Y = read_tsv_files(bases)
  sentenceList = []
  tagList = []
  wordList = []
  annotList = []
  for index, row in X.iterrows():
      docId = row['docId']
      inputText = row['inputText']
      labels = Y.loc[Y.docId == docId]
      sentences = sent_tokenize(inputText)
      for sentence in sentences:
        words = process_sentence(sentence)
        sentence = ' '.join(words);
        if sentence: # if sentence is not empty
          wordList.extend(words)
          sentenceList.append(' '.join(words))
          words_indices,inputText = find_indices(inputText,words)
          annotations = find_annotations(words_indices,labels)
          annotList.extend(annotations)
          tagList.append(' '.join(annotations))
  return sentenceList,tagList,wordList,annotList

In [ ]:
def create_sentences_tags_train(X,Y):
#  bases = getfile_bases()
#  X = read_text_files(bases)
#  Y = read_tsv_files(bases)
  sentenceList = []
  tagList = []
  for index, row in X.iterrows():
      docId = row['docId']
      inputText = row['inputText']
      labels = Y.loc[Y.docId == docId]
      sentences = sent_tokenize(inputText)
      for sentence in sentences:
        words = process_sentence(sentence)
        sentence = ' '.join(words);
        if sentence: # if sentence is not empty
          words_indices,inputText = find_indices(inputText,words)
          annotations = find_annotations_train(words_indices,labels)
          if annotations: # if annot includes quantity
            sentenceList.append(' '.join(words))
            tagList.append(' '.join(annotations))
  return sentenceList,tagList

In [ ]:
def create_train_val_test(X,Y):
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
  X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1)

  file1 = open("/content/tf_ner/data/example/train.words.txt","w") 
  for i in X_train:
    file1.write(i + '\n')
  file1.close()
  file1 = open("/content/tf_ner/data/example/train.tags.txt","w") 
  for i in Y_train:
    file1.write(i + '\n')
  file1.close()
  file1 = open("/content/tf_ner/data/example/val.words.txt","w") 
  for i in X_val:
    file1.write(i + '\n')
  file1.close()
  file1 = open("/content/tf_ner/data/example/val.tags.txt","w") 
  for i in Y_val:
    file1.write(i + '\n')
  file1.close()
  file1 = open("/content/tf_ner/data/example/test.words.txt","w") 
  for i in X_test:
    file1.write(i + '\n')
  file1.close()
  file1 = open("/content/tf_ner/data/example/test.tags.txt","w") 
  for i in Y_test:
    file1.write(i + '\n')
  file1.close()



In [ ]:
def create_train_val_test(X_train,Y_train,X_test,Y_test):
  X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=1)

  file1 = open("/content/tf_ner/data/example/train.words.txt","w") 
  for i in X_train:
    file1.write(i + '\n')
  file1.close()
  file1 = open("/content/tf_ner/data/example/train.tags.txt","w") 
  for i in Y_train:
    file1.write(i + '\n')
  file1.close()
  file1 = open("/content/tf_ner/data/example/val.words.txt","w") 
  for i in X_val:
    file1.write(i + '\n')
  file1.close()
  file1 = open("/content/tf_ner/data/example/val.tags.txt","w") 
  for i in Y_val:
    file1.write(i + '\n')
  file1.close()
  file1 = open("/content/tf_ner/data/example/test.words.txt","w") 
  for i in X_test:
    file1.write(i + '\n')
  file1.close()
  file1 = open("/content/tf_ner/data/example/test.tags.txt","w") 
  for i in Y_test:
    file1.write(i + '\n')
  file1.close()


### Create Sentence Tags and filed

In [ ]:
## find unique indices for each word
def find_indices2(inputText, words):
  words_with_indices = []
  startlist = []
  first = 1
  sentencestart = 0
  end = 0
  for word in words:
    x = inputText.find(word) 
    if x != -1:
      if first == 1:
        sentencestart = x
        first = 0
      start = x
      end =  x + len(word)
      tmp = '!' * len(word)
      inputText = inputText[:start] + tmp + inputText[end:]
      startlist.append(str(start))
      words_with_indices.append((word,start,start+len(word)))
  
  return sentencestart,end,startlist,words_with_indices,inputText

In [ ]:
bases_train = getfile_bases('train')
bases_trial = getfile_bases('trial')

In [ ]:
Y_trainf = read_tsv_files('train',bases_train)
Y_trialf = read_tsv_files('trial',bases_trial)
Y = pd.concat([Y_trainf,Y_trialf],ignore_index=True)

In [ ]:
# A few imports and set up our paths
import itertools
import spacy
import random
import os
from spacy.util import minibatch, compounding

trainpaths = ["/content/MeasEval/data/trial/tsv/",
             "/content/MeasEval/data/train/tsv/"]
evalpath = "/Users/harperco/projects/semeval/measeval-publish-stage/eval/text/"

textpaths = ["/content/MeasEval/data/trial/txt/",
            "/content/MeasEval/data/train/text/"]

In [ ]:
# Collect all the ids and all the text files in both the train and trial directories
# Set our train test split for doing initial model development.
docIds = []
textset = {}
for fileset in textpaths:
    for fn in os.listdir(fileset):
        with open(fileset+fn) as textfile:
            text = textfile.read() #.splitlines()
            #print(fn[:-4])
            textset[fn[:-4]] = text
            docIds.append(fn[:-4])

random.seed(42)
random.shuffle(docIds)

trainIds = docIds[:240]
##evalIds = docIds[:240]
testIds = docIds[240:]

In [ ]:
trainsentenceList = []
traintagList = []
trainstartList = []
testsentenceList = []
testtagList = []
teststartList = []
testdocIds = []
trainwordList = []
trainannotList = []
trainsentenceIdList = []
testwordList = []
testannotList = []
testsentenceIdList = []
sentenceindexList = []
i = 0
j = 0
for fileset in trainpaths:
    for fn in os.listdir(fileset):
        with open(fileset+fn) as annotfile:
            inputText = textset[fn[:-4]]
            next(annotfile)
            labels = Y.loc[Y.docId == fn[:-4]]


            if fn[:-4] in trainIds:
              sentences = sent_tokenize(inputText)
              for sentence in sentences:
                words = process_sentence(sentence)
                sentence = ' '.join(words);
                if sentence: # if sentence is not empty
                  trainsentenceList.append(' '.join(words))
                  Sstart,Send,start,words_indices,inputText = find_indices2(inputText,words)
                  annotations = find_annotations(words_indices,labels)
                  traintagList.append(' '.join(annotations))       
                  trainstartList.append(' '.join(start))
                  sentenceids = [i] * (len(words))
                  trainwordList.extend(words)
                  trainannotList.extend(annotations)
                  trainsentenceIdList.extend(sentenceids)
                  sentenceindexList.append((fn[:-4],Sstart,Send))
                  i+=1
            else:
              sentences = sent_tokenize(inputText)
              for sentence in sentences:
                words = process_sentence(sentence)
                sentence = ' '.join(words);
                if sentence: # if sentence is not empty
                  testsentenceList.append(' '.join(words))
                  Sstart,Send,start,words_indices,inputText = find_indices2(inputText,words)
                  annotations = find_annotations(words_indices,labels)
                  testtagList.append(' '.join(annotations))
                  teststartList.append(' '.join(start))
                  testdocIds.append(fn[:-4]) 
                  sentenceids = [j] * (len(words))
                  testwordList.extend(words)
                  testannotList.extend(annotations)
                  testsentenceIdList.extend(sentenceids)
                  sentenceindexList.append((fn[:-4],Sstart,Send))
                  j+=1   


In [ ]:
df = pd.DataFrame(sentenceindexList, columns=['docId','sentenceStart','sentenceEnd'])

In [ ]:
qesame = 0
qedifferent = 0
qbeforee = 0
qaftere = 0
qpsame = 0
qpdifferent = 0
qbeforep = 0
ebeforep = 0
eafterp = 0
for d in docIds:
  doc = Y.loc[Y.docId == d]
  annotsetlist = doc['annotSet'].unique()
  for a in annotsetlist:
    tmp2 = doc.loc[doc['annotSet'] == a]
    q = tmp2.loc[tmp2.annotType == 'Quantity']
    mentity = tmp2.loc[tmp2.annotType == 'MeasuredEntity']
    mproperty = tmp2.loc[tmp2.annotType == 'MeasuredProperty']

    qafterp = 0
    if not q.empty and not mentity.empty:
      for index2, me in mentity.iterrows():
        same = 0
        for index, row in df.loc[df['docId'] == d].iterrows():
          if (q.iloc[0].startOffset > row.sentenceStart) and (q.iloc[0].startOffset < row.sentenceEnd) and (me.startOffset > row.sentenceStart) and (me.startOffset < row.sentenceEnd):
            qesame+=1
            same = 1
            break
        if same == 0:
          qedifferent +=1
        
        if q.iloc[0].startOffset < me.startOffset:
          qbeforee+=1
        else:
          qaftere +=1

    if not q.empty and not mproperty.empty:
      for index2, mp in mproperty.iterrows():
        same = 0
        for index, row in df.loc[df['docId'] == d].iterrows():
          if (q.iloc[0].startOffset > row.sentenceStart) and (q.iloc[0].startOffset < row.sentenceEnd) and (mp.startOffset > row.sentenceStart) and (mp.startOffset < row.sentenceEnd):
            qpsame+=1
            same = 1
            break
        if same == 0:
          qpdifferent +=1
        
        if q.iloc[0].startOffset < mp.startOffset:
          qbeforep+=1
        else:
          qafterp +=1

    if not mentity.empty and not mproperty.empty:
      for index2, mp in mproperty.iterrows():
        for index3, me in mentity.iterrows():
          if me.startOffset < mp.startOffset:
            ebeforep+=1
          else:
            eafterp +=1

In [ ]:
print('Quantity-Measured Entity Same Sentence ratio' , qesame/(qesame+qedifferent))
print('Quantity-Measured Property Same Sentence ratio' , qpsame/(qpsame+qpdifferent))
print('Mesured Entity coming before  quantitiy ratio' , qaftere/(qaftere+qbeforee))
print('Mesured Property coming after quantitiy ratio' , qbeforep/(qafterp+qbeforep))
print('Mesured Property coming after Measured Entity ratio' , ebeforep/(eafterp+ebeforep))

Quantity-Measured Entity Same Sentence ratio 0.8702090592334495
Quantity-Measured Property Same Sentence ratio 0.9393530997304582
Mesured Entity coming before  quantitiy ratio 0.6820557491289199
Mesured Property coming after quantitiy ratio 0.9939024390243902
Mesured Property coming after Measured Entity ratio 0.6770691994572592


In [ ]:
train_data = pd.DataFrame({'sentence_id':trainsentenceIdList, 'words':trainwordList,'labels':trainannotList})

In [ ]:
test_data = pd.DataFrame({'sentence_id':testsentenceIdList, 'words':testwordList,'labels':testannotList})

### Data preperation

In [ ]:
import shutil

In [ ]:
#shutil.rmtree('/content/cache_dir')

### TRANSFORMER

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import softmax

from simpletransformers.ner import NERModel,NERArgs

In [ ]:
label = train_data["labels"].unique().tolist()

In [ ]:
args = NERArgs()
args.num_train_epochs = 7
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32
args.reprocess_input_data = True

In [ ]:
model = NERModel("bert", "bert-base-cased",labels=label, args=args)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

ValueError: ignored

In [ ]:
model.train_model(train_data)

In [ ]:
result, model_outputs, predictions = model.eval_model(test_data)

In [ ]:
result

In [ ]:
result

{'eval_loss': 0.4618540062354161,
 'f1_score': 0.5355648535564853,
 'precision': 0.5065963060686016,
 'recall': 0.5680473372781065}

In [ ]:
result

{'eval_loss': 0.6114675295984527,
 'f1_score': 0.508,
 'precision': 0.4774436090225564,
 'recall': 0.5427350427350427}

### Create ents{} from prediction

In [ ]:
ents = {}
counts = { "total": 0, "Quantity": 0, "MeasuredEntity": 0, "MeasuredProperty": 0, "Qualifier": 0}
for docId in testdocIds:
  ents[docId] = {}
  for entType in ["Quantity", "MeasuredEntity", "MeasuredProperty", "Qualifier"]:
    ents[docId][entType] = []

In [ ]:
for docId, line,starts,sentence in zip(testdocIds,predictions,teststartList,testsentenceList):
    text = textset[docId]
    counts["total"] += 1
    for tag,start,word in zip(line,starts.split(),sentence.split()):
      for entType in ["Quantity", "MeasuredEntity", "MeasuredProperty", "Qualifier"]:
          if tag == entType:
            ents[docId][entType].append((word,int(start),int(int(start) + len(word))))
            counts[entType]+=1
            counts["total"] += 1

In [ ]:
ents

{'S0006322312001096-1197': {'MeasuredEntity': [('indicators', 14, 24),
   ('General', 154, 161),
   ('Health', 162, 168),
   ('Questionnaire', 169, 182),
   ('(GHQ)', 183, 188),
   ('questionnaire', 317, 330),
   ('response', 376, 384),
   ('categories', 385, 395),
   ('GHQ', 702, 705),
   ('caseness', 706, 714),
   ('major', 782, 787),
   ('depressive', 788, 798),
   ('GHQ', 884, 887),
   ('Whitehall', 969, 978),
   ('II', 979, 981),
   ('study,', 982, 988),
   ('validation', 1068, 1078),
   ('subgroup', 1087, 1095),
   ('GHQ', 1174, 1177),
   ('symptom', 1178, 1185),
   ('caseness', 1186, 1194),
   ('depressive', 1213, 1223),
   ('episodes', 1224, 1232),
   ('psychiatric', 1255, 1266),
   ('interview', 1267, 1276)],
  'MeasuredProperty': [('score', 470, 475),
   ('scores', 547, 553),
   ('sensitive', 721, 730),
   ('specific', 741, 749),
   ('sensitivity', 1005, 1016),
   ('specificity', 1027, 1038),
   ('aged', 1116, 1120),
   ('sensitivity', 1143, 1154),
   ('and', 1155, 1158),
   

### ALL CORRECT LABELED

In [ ]:
ents = {}
counts = { "total": 0, "Quantity": 0, "MeasuredEntity": 0, "MeasuredProperty": 0, "Qualifier": 0}
for docId in testdocIds:
  ents[docId] = {}
  for entType in ["Quantity", "MeasuredEntity", "MeasuredProperty", "Qualifier"]:
    ents[docId][entType] = []

In [ ]:
# Build training data from TSVs in expected format for spacy NER models...
# We have to train each model separately, because spacy doesn't let us have 
# Multiple entities that overlap, and we have this a lot (Especially in our Qualifiers)
# Unfortunately, we even have a fair bit of overlap within annotation types, 
# and end up needing to throw away a bunch of training data.

# Note that we have data split for train / test, and we also have full training data.

trainents = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
traindata = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
testents = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
testdata = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}

alltrainents = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
alltraindata = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}

for fileset in trainpaths:
    for fn in os.listdir(fileset):
        entities = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
        with open(fileset+fn) as annotfile:
            text = textset[fn[:-4]]
            next(annotfile)
            annots = annotfile.read().splitlines()
            for a in annots:
                annot = a.split("\t")
                atype = typemap[annot[2]]
                start = int(annot[3])
                stop = int(annot[4])
                # This is where we toss out the overlaps:
                overlap = False
                for ent in entities[atype]:
                    if ((start >= ent[0] and start <= ent[1]) or (stop >= ent[0] and stop <= ent[1]) or
                        (ent[0] >= start and ent[0] <= stop) or (ent[1] >= start and ent[1] <= stop)):
                        #print(str(start)+"-"+str(stop)+" overlaps " + str(ent))
                        overlap = True
                if overlap == False:    
                    entities[atype].append((start, stop, atype))
                    if fn[:-4] in testIds:
                      ents[fn[:-4]][annot[2]].append((text[start:stop],start,stop))
            if fn[:-4] in trainIds:
                traindata["QUANT"].append((text, {"entities": entities["QUANT"]}))
                traindata["ME"].append((text, {"entities": entities["ME"]}))
                traindata["MP"].append((text, {"entities": entities["MP"]}))
                traindata["QUAL"].append((text, {"entities": entities["QUAL"]}))
                trainents["QUANT"].extend(entities["QUANT"])
                trainents["ME"].extend(entities["ME"])
                trainents["MP"].extend(entities["MP"])
                trainents["QUAL"].extend(entities["QUAL"])
            else:
                testdata["QUANT"].append((text, {"entities": entities["QUANT"]}))
                testdata["ME"].append((text, {"entities": entities["ME"]}))
                testdata["MP"].append((text, {"entities": entities["MP"]}))
                testdata["QUAL"].append((text, {"entities": entities["QUAL"]}))
                testents["QUANT"].extend(entities["QUANT"])
                testents["ME"].extend(entities["ME"])
                testents["MP"].extend(entities["MP"])
                testents["QUAL"].extend(entities["QUAL"])
            alltraindata["QUANT"].append((text, {"entities": entities["QUANT"]}))
            alltraindata["ME"].append((text, {"entities": entities["ME"]}))
            alltraindata["MP"].append((text, {"entities": entities["MP"]}))
            alltraindata["QUAL"].append((text, {"entities": entities["QUAL"]}))
            alltrainents["QUANT"].extend(entities["QUANT"])
            alltrainents["ME"].extend(entities["ME"])
            alltrainents["MP"].extend(entities["MP"])
            alltrainents["QUAL"].extend(entities["QUAL"])

In [ ]:
ents['S0019103512003533-5598']

### Process ent

In [ ]:
for docId, allents in ents.items():
  for entType in ["Quantity", "MeasuredEntity", "MeasuredProperty", "Qualifier"]:
    tmp = allents[entType]
    removelist = []
    for i in range(len(tmp)-1):
      if tmp[i+1][1] - tmp[i][2] < 2:
        tmp[i+1] = (tmp[i][0] + ' ' + tmp[i+1][0],tmp[i][1], tmp[i+1][2])
        removelist.append(i)
    for i in sorted(removelist, reverse=True):
      tmp.pop(i)

### ENt to TSV

#### Preperation

In [ ]:
# A few imports and set up our paths
import itertools
import spacy
import random
import os
from spacy.util import minibatch, compounding

trainpaths = ["/content/MeasEval/data/trial/tsv/",
             "/content/MeasEval/data/train/tsv/"]
evalpath = "/Users/harperco/projects/semeval/measeval-publish-stage/eval/text/"

textpaths = ["/content/MeasEval/data/trial/txt/",
            "/content/MeasEval/data/train/text/"]

In [ ]:
# Set shorthands for annotation spans
typemap = {"Quantity": "QUANT",
           "MeasuredEntity": "ME", 
           "MeasuredProperty": "MP", 
           "Qualifier": "QUAL"}

In [ ]:
# Collect a set of unique units for use in populating the unit data...
import json
units = []

for fileset in trainpaths:
    for fn in os.listdir(fileset):
        # Let's make sure to limit the units to just the smaller train set
        if fn[:-4] in trainIds:
            with open(fileset+fn) as annotfile:
                text = textset[fn[:-4]]
                next(annotfile)
                annots = annotfile.read().splitlines()
                for a in annots:
                    annot = a.split("\t")
                    atype = typemap[annot[2]]
                    if atype == "QUANT" and annot[7] != "":
                        jsondata = json.loads(annot[7])
                        if "unit" in jsondata:
                            units.append(jsondata["unit"])
uniqunits = list(set(units))

In [ ]:
%mkdir /content/MeasEval/baseline-simpler-split

mkdir: cannot create directory ‘/content/MeasEval/baseline-simpler-split’: File exists


#### Baseline2

In [ ]:
# This last, fairly unweildy chunk of code is:
# * collecting everything, 
# * Building the TSV strings
# * Attempting to identify a unit
# * matching and populating annotSet based on knockout logic, 
# * resorting, and populating TSV files.

# Configure header string and submission directory (latter needs to exist.)
header = "docId\tannotSet\tannotType\tstartOffset\tendOffset\tannotId\ttext\tother"
subdir = "/content/MeasEval/baseline-simpler-split/"

for docId, allents in ents.items():
    # First we collect our Quantities
    # We want to get the strin version, the full set, and the "knockout" list.
    quantstrings = []
    quants = []
    knockout = []
    annotSet = 1
    for quant in allents['Quantity']:
        # Match units in the Quant, then find the longest unit 
        unitmatches = []
        for unit in uniqunits: 
            if unit in quant[0]:
                unitmatches.append(unit)
        if len(unitmatches) > 0: 
            unit = max(unitmatches, key=len)
        # Build the quantity string, and also the dictionary for quant and knockout.
        quantstrings.append(docId + "\t" + str(annotSet) + "\tQuantity\t" + str(quant[1]) + "\t" +
                          str(quant[2]) + "\t1\t" + quant[0]+"\t{\"unit\": \"" + unit +  "\"}")
        quants.append({"annotSet": annotSet, "annotId": 1, "start": quant[1], "end": quant[2], 
                       "text": quant[0], "type": "Quantity"}) 
        knockout.append({"annotSet": annotSet, "annotId": 1, "start": quant[1], "end": quant[2], 
                       "text": quant[0], "type": "Quantity"}) 
        annotSet+=1
    
    # So now we want to do the ents, as we need this queued up to do more matching with the MPs
    mestrings = []
    mestring = ""
    mes = []
    knockoutmes = []
    #annotSet = 1
    for me in allents['MeasuredEntity']:
        knockoutmes.append({"start": me[1], "end": me[2], "text": me[0], "type": "MeasuredEntity"}) 

    # Now we work through our measured properties.
    mpstrings = []
    mpstring = ""
    mps = []
    knockoutmps = []
    for mp in allents['MeasuredProperty']:
        if len(knockout) > 0 and len(knockoutmes) > 0:
        start = mp[1]
        end = mp[2]
        nearest = {"dist": 100000000, "set": 0, "id": 0, "index": 100000000}
        index = 0
        qcopy = []
        find = 0
        removeq = knockout[0]
        for q in knockout:
            for ix, row in df.loc[df['docId'] == docId].iterrows():
              if (q["start"] > row.sentenceStart) and (q["start"] < row.sentenceEnd) and (start > row.sentenceStart) and (start < row.sentenceEnd):
                find = 1
                qcopy.append(q)
        if find == 1:
          for qc in qcopy:
            dists = [abs(start-qc["start"]), abs(end-qc["start"]), abs(start-qc["end"]), abs(end-qc["end"])]
            mindist = min(dists)
            if mindist < nearest["dist"]:
                nearest["dist"] = mindist
                nearest["set"] = qc["annotSet"]
                nearest["id"] = qc["annotId"]
                nearest["index"] = index
                removeq = qc
        else:
          for q in knockout:
                dists = [abs(start-q["start"]), abs(end-q["start"]), abs(start-q["end"]), abs(end-q["end"])]
                mindist = min(dists)
                if mindist < nearest["dist"]:
                    nearest["dist"] = mindist
                    nearest["set"] = q["annotSet"]
                    nearest["id"] = q["annotId"]
                    nearest["index"] = index
                    removeq = q
          ##       index+=1
        ##knockout.pop(nearest["index"])
        knockout.remove(removeq)

        mpString = (docId + "\t" + str(nearest["set"]) + "\tMeasuredProperty\t" + str(mp[1]) + "\t" + 
                    str(mp[2]) + "\t" + str(nearest["id"]+1) + "\t" + mp[0] + "\t{\"HasQuantity\": \"" + 
                    str(nearest["id"]) + "\"}" )
        mpstrings.append(mpString)
        mps.append({"annotSet": nearest["set"], "annotId": nearest["id"]+1, "start": mp[1], 
                    "end": mp[2], "text": mp[0], "type": "MeasuredProperty"})
        knockoutmps.append({"annotSet": nearest["set"], "annotId": nearest["id"]+1, "start": mp[1], 
                    "end": mp[2], "text": mp[0], "type": "MeasuredProperty"})

        nearestme = {"dist": 100000000, "index": 100000000}
        index = 0
        meremove = knockoutmes[0]
        if len(knockoutmes) > 0:
            for me in knockoutmes:
                dists = [abs(start-me["start"]), abs(end-me["start"]), abs(start-me["end"]), abs(end-me["end"])]
                mindist = min(dists)
                if mindist < nearestme["dist"]:
                    nearestme["dist"] = mindist
                    nearestme["index"] = index
                    meremove = me
                index+=1
            
            meString = (docId + "\t" + str(nearest["set"]) + "\tMeasuredEntity\t" + str(meremove["start"]) + "\t" + 
                        str(meremove["end"]) + "\t" + str(nearest["id"]+2) + "\t" + meremove["text"] + "\t{\"HasProperty\": \"" + 
                        str(nearest["id"]+1) + "\"}" )   
            mestrings.append(meString)

               ## knockoutmes.pop(nearestme["index"])


    # Now we do any leftover MEs, which should go straight to a Quantity:

    for me in knockoutmes:
        start = me["start"]
        end = me["end"]
        nearest = {"dist": 100000000, "set": 0, "id": 0, "index": 100000000, "type": ""}
        index = 0
        qcopy =[]
        find = 0
        if len(knockout) > 0:
          for q in knockout:
            for ix, row in df.loc[df['docId'] == docId].iterrows():
              if (q["start"] > row.sentenceStart) and (q["start"] < row.sentenceEnd) and (start > row.sentenceStart) and (start < row.sentenceEnd):
                find = 1
                qcopy.append(q)
          removeq = knockout[0]
          if find == 1:
            for qc in qcopy:
              dists = [abs(start-qc["start"]), abs(end-qc["start"]), abs(start-qc["end"]), abs(end-qc["end"])]
              mindist = min(dists)
              if mindist < nearest["dist"]:
                  nearest["dist"] = mindist
                  nearest["set"] = qc["annotSet"]
                  nearest["id"] = qc["annotId"]
                  nearest["index"] = index
                  removeq = qc
          else:
            for q in knockout:
                dists = [abs(start-q["start"]), abs(end-q["start"]), abs(start-q["end"]), abs(end-q["end"])]
                mindist = min(dists)
                if mindist < nearest["dist"]:
                    nearest["dist"] = mindist
                    nearest["set"] = q["annotSet"]
                    nearest["id"] = q["annotId"]
                    nearest["index"] = index
                    nearest["type"] = q["type"]
                    removeq = qc
                index+=1

         ## knockout.pop(nearest["index"])
          knockout.remove(removeq)
          meString = (docId + "\t" + str(nearest["set"]) + "\tMeasuredEntity\t" + str(me["start"]) + "\t" + 
                      str(me["end"]) + "\t" + str(nearest["id"]+1) + "\t" + me["text"] + "\t{\"HasQuantity\": \"" + 
                      str(nearest["id"]) + "\"}" )   
          mestrings.append(meString)
          mes.append({"annotSet": nearest["set"], "annotId": nearest["id"]+1, "start": me["start"], 
                      "end": me["end"], "text": me["text"], "type": "MeasuredEntity"})
            
    #Finally, let's process our Qualifiers:
    kitchensink = [x for x in itertools.chain(quants, mps, mes)]
    qualstrings = []
    for qual in allents['Qualifier']:
        start = qual[1]
        end = qual[2]
        nearest = {"dist": 100000000, "set": 0, "id": 0, "index": 100000000}
        index = 0
        for q in kitchensink:
            dists = [abs(start-q["start"]), abs(end-q["start"]), abs(start-q["end"]), abs(end-q["end"])]
            mindist = min(dists)
            if mindist < nearest["dist"]:
                nearest["dist"] = mindist
                nearest["set"] = q["annotSet"]
                nearest["id"] = q["annotId"]
                nearest["index"] = index
            index+=1
        if kitchensink:
          kitchensink.pop(nearest["index"])

        qualString = (docId + "\t" + str(nearest["set"]) + "\tQualifier\t" + str(qual[1]) + "\t" + 
                    str(qual[2]) + "\t" + str(nearest["id"]+1) + "\t" + qual[0] + "\t{\"Qualifies\": \"" + 
                    str(nearest["id"]) + "\"}" )
        qualstrings.append(qualString)

    # Finally, we collect everythign:

    import itertools
    allstrings = [x for x in itertools.chain(quantstrings, mpstrings, mestrings, qualstrings)]
    sortedstrings = {}

    sub = open(subdir+docId + ".tsv", "w")

    for string in allstrings:
        annotSet = string.split("\t")[1]
        annotId = string.split("\t")[5]
        if annotSet not in sortedstrings:
            sortedstrings[annotSet] = {}
        sortedstrings[annotSet][annotId] = string   
    sub.write(header+"\n")
    for aset, val in sortedstrings.items():
        for aid, string in val.items():
            sub.write(string+"\n")
    sub.close()

#### f1 0.30

Applied changes on the given baseline to create realtionship among detected entities in the text.

In [ ]:
# This last, fairly unweildy chunk of code is:
# * collecting everything, 
# * Building the TSV strings
# * Attempting to identify a unit
# * matching and populating annotSet based on knockout logic, 
# * resorting, and populating TSV files.

# Configure header string and submission directory (latter needs to exist.)
header = "docId\tannotSet\tannotType\tstartOffset\tendOffset\tannotId\ttext\tother"
subdir = "/content/MeasEval/baseline-simpler-split/"

for docId, allents in ents.items():
    # First we collect our Quantities
    # We want to get the strin version, the full set, and the "knockout" list.
    quantstrings = []
    quants = []
    knockout = []
    annotSet = 1
    for quant in allents['Quantity']:
        # Match units in the Quant, then find the longest unit 
        unitmatches = []
        for unit in uniqunits: 
            if unit in quant[0]:
                unitmatches.append(unit)
        if len(unitmatches) > 0: 
            unit = max(unitmatches, key=len)
        # Build the quantity string, and also the dictionary for quant and knockout.
        quantstrings.append(docId + "\t" + str(annotSet) + "\tQuantity\t" + str(quant[1]) + "\t" +
                          str(quant[2]) + "\t1\t" + quant[0]+"\t{\"unit\": \"" + unit +  "\"}")
        quants.append({"annotSet": annotSet, "annotId": 1, "start": quant[1], "end": quant[2], 
                       "text": quant[0], "type": "Quantity"}) 
        knockout.append({"annotSet": annotSet, "annotId": 1, "start": quant[1], "end": quant[2], 
                       "text": quant[0], "type": "Quantity"}) 
        annotSet+=1
    
    # So now we want to do the ents, as we need this queued up to do more matching with the MPs
    mestrings = []
    mestring = ""
    mes = []
    knockoutmes = []
    #annotSet = 1
    for me in allents['MeasuredEntity']:
        knockoutmes.append({"start": me[1], "end": me[2], "text": me[0], "type": "MeasuredEntity"}) 

    # Now we work through our measured properties.
    mpstrings = []
    mpstring = ""
    mps = []
    knockoutmps = []
    for mp in allents['MeasuredProperty']:
        if len(knockout) > 0 and len(knockoutmes) > 0:
            start = mp[1]
            end = mp[2]
            nearest = {"dist": 100000000, "set": 0, "id": 0, "index": 100000000}
            index = 0
            for q in knockout:
                dists = [abs(start-q["start"]), abs(end-q["start"]), abs(start-q["end"]), abs(end-q["end"])]
                mindist = min(dists)
                if mindist < nearest["dist"]:
                    nearest["dist"] = mindist
                    nearest["set"] = q["annotSet"]
                    nearest["id"] = q["annotId"]
                    nearest["index"] = index
                index+=1
            knockout.pop(nearest["index"])

            mpString = (docId + "\t" + str(nearest["set"]) + "\tMeasuredProperty\t" + str(mp[1]) + "\t" + 
                        str(mp[2]) + "\t" + str(nearest["id"]+1) + "\t" + mp[0] + "\t{\"HasQuantity\": \"" + 
                        str(nearest["id"]) + "\"}" )
            mpstrings.append(mpString)
            mps.append({"annotSet": nearest["set"], "annotId": nearest["id"]+1, "start": mp[1], 
                        "end": mp[2], "text": mp[0], "type": "MeasuredProperty"})
            knockoutmps.append({"annotSet": nearest["set"], "annotId": nearest["id"]+1, "start": mp[1], 
                        "end": mp[2], "text": mp[0], "type": "MeasuredProperty"})




    # Now we do any leftover MEs, which should go straight to a Quantity:

    for me in knockoutmes:
        start = me["start"]
        end = me["end"]
        nearest = {"dist": 100000000, "set": 0, "id": 0, "index": 100000000, "type": ""}
        index = 0                
        for q in knockout:
            dists = [abs(start-q["start"]), abs(end-q["start"]), abs(start-q["end"]), abs(end-q["end"])]
            mindist = min(dists)
            if mindist < nearest["dist"]:
                nearest["dist"] = mindist
                nearest["set"] = q["annotSet"]
                nearest["id"] = q["annotId"]
                nearest["index"] = index
                nearest["type"] = q["type"]
            index+=1
        if len(knockout) > 0:
            knockout.pop(nearest["index"])
            meString = (docId + "\t" + str(nearest["set"]) + "\tMeasuredEntity\t" + str(me["start"]) + "\t" + 
                        str(me["end"]) + "\t" + str(nearest["id"]+1) + "\t" + me["text"] + "\t{\"HasQuantity\": \"" + 
                        str(nearest["id"]) + "\"}" )   
            mestrings.append(meString)
            mes.append({"annotSet": nearest["set"], "annotId": nearest["id"]+1, "start": me["start"], 
                        "end": me["end"], "text": me["text"], "type": "MeasuredEntity"})
            
    #Finally, let's process our Qualifiers:
    kitchensink = [x for x in itertools.chain(quants, mps, mes)]
    qualstrings = []
    for qual in allents['Qualifier']:
        start = qual[1]
        end = qual[2]
        nearest = {"dist": 100000000, "set": 0, "id": 0, "index": 100000000}
        index = 0
        for q in kitchensink:
            dists = [abs(start-q["start"]), abs(end-q["start"]), abs(start-q["end"]), abs(end-q["end"])]
            mindist = min(dists)
            if mindist < nearest["dist"]:
                nearest["dist"] = mindist
                nearest["set"] = q["annotSet"]
                nearest["id"] = q["annotId"]
                nearest["index"] = index
            index+=1
        if kitchensink:
          kitchensink.pop(nearest["index"])

        qualString = (docId + "\t" + str(nearest["set"]) + "\tQualifier\t" + str(qual[1]) + "\t" + 
                    str(qual[2]) + "\t" + str(nearest["id"]+1) + "\t" + qual[0] + "\t{\"Qualifies\": \"" + 
                    str(nearest["id"]) + "\"}" )
        qualstrings.append(qualString)

    # Finally, we collect everythign:

    import itertools
    allstrings = [x for x in itertools.chain(quantstrings, mpstrings, mestrings, qualstrings)]
    sortedstrings = {}

    sub = open(subdir+docId + ".tsv", "w")

    for string in allstrings:
        annotSet = string.split("\t")[1]
        annotId = string.split("\t")[5]
        if annotSet not in sortedstrings:
            sortedstrings[annotSet] = {}
        sortedstrings[annotSet][annotId] = string   
    sub.write(header+"\n")
    for aset, val in sortedstrings.items():
        for aid, string in val.items():
            sub.write(string+"\n")
    sub.close()

### Evaluation Script

Evaluates the created tsv files using a script written by the task organizers.

In [ ]:
from distutils.dir_util import copy_tree

In [ ]:
%cd MeasEval
!mkdir gold
%cd ..

/content/MeasEval
/content


In [ ]:
copy_tree("/content/MeasEval/data/train/tsv/","/content/MeasEval/gold/")
copy_tree("/content/MeasEval/data/trial/tsv/","/content/MeasEval/gold/")

['/content/MeasEval/gold/S0022000014000026-17824.tsv',
 '/content/MeasEval/gold/S0019103511004994-1511.tsv',
 '/content/MeasEval/gold/S0019103512002801-1927.tsv',
 '/content/MeasEval/gold/S0021979713004438-1969.tsv',
 '/content/MeasEval/gold/S0016236113008041-967.tsv',
 '/content/MeasEval/gold/S0012821X13007309-1509.tsv',
 '/content/MeasEval/gold/S0012821X12004384-1415.tsv',
 '/content/MeasEval/gold/S0012821X13002185-1217.tsv',
 '/content/MeasEval/gold/S0012821X12004384-1405.tsv',
 '/content/MeasEval/gold/S0012821X13002185-1231.tsv',
 '/content/MeasEval/gold/S0016236113008041-3153.tsv',
 '/content/MeasEval/gold/S0019103513005058-1737.tsv',
 '/content/MeasEval/gold/S0016236113008041-3269.tsv',
 '/content/MeasEval/gold/S0019103512003533-5031.tsv',
 '/content/MeasEval/gold/S0012821X12004384-1302.tsv',
 '/content/MeasEval/gold/S0019103512003533-5251.tsv',
 '/content/MeasEval/gold/S0019103512003995-1910.tsv',
 '/content/MeasEval/gold/S0019103512002801-1824.tsv',
 '/content/MeasEval/gold/S00

In [ ]:
!pip install -r /content/MeasEval/eval/requirements.txt

  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26820 sha256=244256aacb95b4229d219d29cd2ce2fd2e0cea589a69c0b55ced82d25d0baf8c
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [ ]:
!python /content/MeasEval/eval/measeval-eval.py -i "/content/MeasEval/" -g "gold/" -s "baseline-simpler-split/" -l


Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S2213671113000738-435.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S1873506113001116-710.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0927024813002420-975.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S1359835X13001875-1359.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0167577X14001256-389.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S175058361300203X-1556.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0921818113002245-882.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0006322312001096-1221.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler

In [ ]:
!python /content/MeasEval/eval/measeval-eval.py -i "/content/MeasEval/" -g "gold/" -s "baseline-simpler-split/" -l


Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S2213671113000738-435.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S1873506113001116-710.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0927024813002420-975.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S1359835X13001875-1359.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0167577X14001256-389.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S175058361300203X-1556.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0921818113002245-882.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0006322312001096-1221.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler

In [ ]:
!python /content/MeasEval/eval/measeval-eval.py -i "/content/MeasEval/" -g "gold/" -s "baseline-simpler-split/" -m "class" -l 


Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S2213671113000738-435.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S1873506113001116-710.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0927024813002420-975.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S1359835X13001875-1359.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0167577X14001256-389.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S175058361300203X-1556.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0921818113002245-882.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler-split/S0006322312001096-1221.tsv'))
Passed! :)

Validating Vlad(source=LocalFile('/content/MeasEval/baseline-simpler